In [14]:
import numpy as np
import ModeloBase
from ModeloBase import read_data_model, add_laplace_noise, rmse
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as r2
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor  # Importación del modelo XGBoost
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# Cargar los datos de California Housing
all_data = fetch_california_housing()
X_train_val, X_val, X_test, Y_train_val, Y_val, Y_test = read_data_model(all_data, 'MedInc')

# Entrenar el escalador con los datos train_val para evitar fuga de datos antes del entrenamiento
scaler = StandardScaler()
X_train_val_scaled = scaler.fit_transform(X_train_val)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Definir los parámetros para la búsqueda en cuadrícula
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Configurar el modelo y la búsqueda en cuadrícula
xgbr = XGBRegressor(objective='reg:squarederror', random_state=42)
grid_search = GridSearchCV(
    estimator=xgbr,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Entrenar el modelo con la búsqueda de hiperparámetros
grid_search.fit(X_train_val_scaled, Y_train_val)
best_params = grid_search.best_params_

# Imprimir los mejores parámetros encontrados
print("Best parameters found:", best_params)

# Entrenamiento del modelo original sin ruido con los mejores parámetros
model_original = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
model_original.fit(X_train_val_scaled, Y_train_val)
Y_predict = model_original.predict(X_test_scaled)

print(f"X_test : {X_test}")
print(f"Y_test : {Y_test}")
print(f"Y_predict : {Y_predict}")

# Introducción de ruido óptimo en los datos
epsilon_optimo = 1
np.random.seed(0)  # Semilla para reproducibilidad

X_train_val_noisy = add_laplace_noise(X_train_val, epsilon_optimo)
X_test_noisy = add_laplace_noise(X_test, epsilon_optimo)

scalerRuido = StandardScaler()
X_train_val_noisy_scaled = scalerRuido.fit_transform(X_train_val_noisy)
X_test_noisy_scaled = scalerRuido.transform(X_test_noisy)

# Entrenamiento del modelo con los datos ruidosos
model_noisy = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
model_noisy.fit(X_train_val_noisy_scaled, Y_train_val)
Y_predict_optimo = model_noisy.predict(X_test_noisy_scaled)

# Definir 20 niveles de ruido adicionales que el atacante probará usando Laplace
additional_epsilons = np.logspace(np.log10(0.01), np.log10(1000), num=50)
additional_epsilons = np.insert(additional_epsilons, 0, 0)

print("\nResultados del ataque para los primeros 10 valores:")

numero_muestras = 50

X_test_reducido = X_test[0:numero_muestras, 0]
X_test_noisy_reducido = X_test_noisy[0:numero_muestras, 0]

MAE_delivery = mae(X_test_reducido, X_test_noisy_reducido)
RMSE_delivery = rmse(X_test_reducido, X_test_noisy_reducido)
R2_delivery = r2(X_test_reducido, X_test_noisy_reducido)

print("\nerror en la entrega:")
print(f"MAE: {MAE_delivery}")
print(f"RMSE: {RMSE_delivery}")
print(f"R2: {R2_delivery}")


# Inicializar listas para estadísticas detalladas
improvement = []
worsening = []

# Inicializar contadores para mejoras y empeoramientos
count_improvements = 0
count_worsenings = 0

X_test_attack = np.empty(numero_muestras)

# Simulación del ataque para los primeros 10 valores
for i in range(numero_muestras):
    original_noisy_value = X_test_noisy[i, 0]  # Valor con ruido
    original_value = X_test[i, 0]  # Valor original sin ruido
    print(f"\nValor original sin ruido (index {i}): {original_value}")
    print(f"Valor ruidoso original (index {i}): {original_noisy_value}")
    print(f"Y_test original (index {i}): {Y_test[i]}")
    print(f"Y Predict original (index {i}): {Y_predict[i]}")
    print(f"Y Predict optimo (index {i}): {Y_predict_optimo[i]}")
    print(f" Error prediccion (index {i}): {abs(Y_test[i] - Y_predict_optimo[i])}")

    best_error = float('inf')
    best_prediction = None
    best_epsilon = None

    for epsilon in additional_epsilons:
        # Generar ruido adicional usando Laplace
        if (epsilon != 0):
            additional_noise_test = add_laplace_noise(X_test_noisy, epsilon)
        else:
            additional_noise_test = X_test_noisy.copy()

        additional_noise_test_scaled = scaler.transform(additional_noise_test)

        # Ajustar el valor de entrada con el nuevo ruido
        noisy_X_test_scaled = [additional_noise_test_scaled[i, :]]  # Suponiendo que la columna de interés es la primera

        # Realizar la predicción para esta fila
        prediction = model_original.predict(noisy_X_test_scaled)

        # Calcular el error
        true_value = Y_test[i]
        error = abs(true_value - prediction)

        # Mostrar el valor con ruido añadido y la predicción correspondiente
        print(f"  - Valor ajustado: {additional_noise_test[i, 0]}, Predicción: {prediction}, Epsilon usado: {epsilon}, Error: {error}")

        # Evaluar si este error es el mejor (menor)
        if error < best_error:
            best_error = error
            best_prediction = prediction
            best_epsilon = epsilon

            X_test_attack[i] = additional_noise_test[i, 0]

    print(f"Mejor X: {X_test_attack[i]} Mejor Epsilon para index {i}: {best_epsilon}, Mejor Error: {best_error}, Mejor Predicción: {best_prediction}")
    error_x_noise = abs(X_test_noisy[i, 0] - X_test[i, 0])
    error_x_attack = abs(X_test_attack[i] - X_test[i, 0])

    improvement_value = error_x_noise - error_x_attack
    improvement.append(improvement_value)
    if improvement_value > 0:
        count_improvements += 1
    else:
        worsening_value = error_x_attack - error_x_noise
        worsening.append(worsening_value)
        if worsening_value > 0:
            count_worsenings += 1

    print(f"mejora {improvement_value}  R pre {error_x_noise} R pos {error_x_attack}")

MAE_attack = mae(X_test_reducido, X_test_attack)
RMSE_attack = rmse(X_test_reducido, X_test_attack)
R2_attack = r2(X_test_reducido, X_test_attack)

print("\nerror en la entrega:")
print(f"MAE: {MAE_delivery}")
print(f"RMSE: {RMSE_delivery}")
print(f"R2: {R2_delivery}")

print("\nerror attack:")
print(f"MAE: {MAE_attack}")
print(f"RMSE: {RMSE_attack}")
print(f"R2: {R2_attack}")


# Estadísticas detalladas
improvement = np.array(improvement)
worsening = np.array(worsening)


# Filtrar solo mejoras positivas
positive_improvements = improvement[improvement > 0]


print("\nEstadísticas detalladas de mejora:")
print(f"Mejora Media: {np.mean(positive_improvements)}")
print(f"Varianza de la Mejora: {np.var(positive_improvements)}")
print(f"Número de puntos que mejoran: {count_improvements}")
print(f"Mejora Máxima: {np.max(positive_improvements)}")
print(f"Mejora Mínima: {np.min(positive_improvements)}")

print("\nEstadísticas detalladas de empeoramiento:")
print(f"Empeoramiento Medio: {np.mean(worsening)}")
print(f"Varianza del Empeoramiento: {np.var(worsening)}")
print(f"Número de puntos que empeoran: {count_worsenings}")
print(f"Empeoramiento Máximo: {np.max(worsening)}")
print(f"Empeoramiento Mínimo: {np.min(worsening)}")


Y_data: 20640
y_data: 19648
y_data: 18665
Best parameters found: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
X_test : [[   3.463         8.            6.36363636 ...    2.89800443
    39.08       -121.04      ]
 [   3.2891       27.            5.24033149 ...    2.42541436
    38.05       -122.14      ]
 [   2.096        34.            2.46689304 ...    3.37351443
    34.05       -118.3       ]
 ...
 [   6.4415       15.            7.07798165 ...    2.89449541
    33.83       -117.76      ]
 [   2.425        17.            3.16008772 ...    1.68640351
    33.89       -117.87      ]
 [   3.0938       32.            4.71764706 ...    2.23921569
    38.25       -122.04      ]]
Y_test : [2.017 1.506 1.875 ... 3.007 1.717 1.104]
Y_predict : [1.5475898 1.401996  1.9830345 ... 2.8735013 1.3602865 1.6196193]

Resultados del ataque para los primeros 10 valores:

error en la entrega:
MAE: 13.138027078018375
RMSE: 17.759378930395194
R2: -1